# Декораторы функций

## Самое простое исполнение

Концепция: пусть у нас есть некоторая функция `simple_decorator`, параметром которой является ссылка на некоторую другую функцию.
Самая простая реализация декоратора - когда мы подменяем ссылку на исходную функцию `func` на функцию `wrapper`. 
Вот как это выглядит:

In [2]:
def func_decorator(func):
    def wrapper():
        print("--- Оборотка до функции ---")
        func()
        print("--- Оборотка после функции ---")

    return wrapper


def my_func():
    print("Вызов исходной функции my_func")


my_func()

Вызов исходной функции my_func


Запишем в переменную `new_my_func` ссылку на функцию **`wrapper`**.

In [5]:
new_my_func = func_decorator(my_func)
new_my_func()

--- Оборотка до функции ---
Вызов исходной функции my_func
--- Оборотка после функции ---


Как видим, мы вызвали не сам **`my_func`**, а обертку **`wrapper`**.

Но обычно поступают проще, просто переименовывая **`my_func`**:

In [7]:
my_func = func_decorator(my_func)
my_func()

--- Оборотка до функции ---
--- Оборотка до функции ---
Вызов исходной функции my_func
--- Оборотка после функции ---
--- Оборотка после функции ---


**Замечание**: если мы немного поменяем функцию **`my_func`**, например, добавив в нее параметр, то работать это уже не будет:

In [16]:
def my_func2(param):
    print(f"Вызов исходной функции my_func c параметром {param}")


my_func2 = func_decorator(my_func2)
my_func2("param")

TypeError: func_decorator.<locals>.wrapper() takes 0 positional arguments but 1 was given

Для *избегания* этой ошибки просто функции **`wrapper`** предоставляем нужный параметр:

In [22]:
def func_decorator2(func):
    def wrapper2(param):
        print("--- Обработка до функции ---")
        func(param)
        print("--- Обработка после функции ---")

    return wrapper2


my_func2 = func_decorator2(my_func2)
my_func2("param")

--- Обработка до функции ---
--- Обработка до функции ---
--- Обработка до функции ---
--- Обработка до функции ---
--- Обработка до функции ---
--- Обработка до функции ---


TypeError: func_decorator.<locals>.wrapper() takes 0 positional arguments but 1 was given

И все равно не на всех версиях `Python` это сработает. Тогда сделаем универсальный способ декорирования функций:

In [24]:
def universal_decorator(func):
    def wrapper(*args, **kwargs):
        print("--- Обработка до функции ---")
        func(*args, **kwargs)
        print("--- Обработка после функции ---")

    return wrapper


def my_func(x, y, z, t):
    print(x + y + z + t)


my_func = universal_decorator(my_func)
my_func(1, 2, 3, 4)

--- Обработка до функции ---
10
--- Обработка после функции ---


Последний штрих: если **`my_func`** будет что-то возвращать, то опять все сломается:

In [26]:
def my_func(x, y, z, t):
    print("Вызов функции my_func")
    return x + y + z + t


my_func = universal_decorator(my_func)
result = my_func(1, 2, 3, 4)
print(result)

--- Обработка до функции ---
Вызов функции my_func
--- Обработка после функции ---
None


Напишем тогда декоратор и для такого случая:

In [35]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print("--- Обработка до функции ---")
        result = func(*args, **kwargs)
        print("--- Обработка после функции ---")
        return result

    return wrapper


def my_func(x, y, z, t):
    print("Вызов функции my_func")
    return x + y + z + t


my_func = decorator(my_func)
print(my_func(1, 2, 3, 4))

--- Обработка до функции ---
Вызов функции my_func
--- Обработка после функции ---
10


### Смысл написанного выше кода

Представим, что нам нужно протестировать функции на скорость их выполнения.

Давайте напишем функцию, вычисляющую наибольший общий делитель (медленный алгоритм Евклида):

In [40]:
def nod(a, b):
    while a != b:
        if a > b:
            a -= b
        else:
            b -= a
    return a


print(nod(7, 11))
print(nod(24, 6))

1
6


И декоратор-секундомер к нему:

In [42]:
import time


def stop_watch(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        finish = time.time()
        delta = finish - start
        print(f"Время работы функции: {delta}")
        return result

    return wrapper


time_nod = stop_watch(nod)
print(time_nod(7, 11))
print(time_nod(5672348, 32147329))
print(time_nod(2, 100000000))

Время работы функции: 9.059906005859375e-06
1
Время работы функции: 2.288818359375e-05
1
Время работы функции: 4.573429107666016
2


На реальной практике этой херней с переименованием ссылки на функцию никто не занимается. В современном `Python` все пишут что-то такое:

In [43]:
def stop_watch(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        finish = time.time()
        delta = finish - start
        print(f"Время работы функции: {delta}")
        return result

    return wrapper


@stop_watch
def nod(a, b):
    while a != b:
        if a > b:
            a -= b
        else:
            b -= a
    return a


print(nod(7, 11))
print(nod(5672348, 32147329))
print(nod(2, 100000000))

Время работы функции: 1.4781951904296875e-05
1
Время работы функции: 2.193450927734375e-05
1
Время работы функции: 4.882281064987183
2


## Декораторы посложнее. Декораторы с параметром

Пусть мы хотим создать декоратор, вычисляющий производную некоторой математической функции. Пусть это будет $f(x) = x^2*e^x$.

В самом простом исполнении это выглядит следующим образом:

In [46]:
import math


def der(func):
    def wrapper(x, *args, **kwargs):
        dx = 0.0001
        der = (func(x + dx, *args, **kwargs) - func(x, *args, **kwargs)) / dx
        return der

    return wrapper


@der
def f(x):
    return x * x * math.exp(x)


# Напомню, что f'(x) = (x^2*exp(x))' = x*(x+2)*exp(x)

print(f(0))
print(f(1))
print(f(1) / math.exp(1))

0.00010001000050001668
8.155796942914684
3.0003500216672117


Все получилось, но точность может нас не всегда устраивать. Точность здесь задается переменной `dx`: чем она меньше, тем точность больше. Давайте попробуем отрегулировать точность:

In [47]:
def der(func, dx=0.0001):
    def wrapper(x, *args, **kwargs):
        der = (func(x + dx, *args, **kwargs) - func(x, *args, **kwargs)) / dx
        return der

    return wrapper


@der(dx=0.1)
def f(x):
    return x * x * math.exp(x)


print(f(0))

TypeError: der() missing 1 required positional argument: 'func'

Не работает :( . В `Python` правильно писать следующий синтаксис, обернув еще в одну функцию:

In [52]:
def MAIN_DECORATOR(dx=0.0001):
    def der(func):
        def wrapper(x, *args, **kwargs):
            der = (func(x + dx, *args, **kwargs) -
                   func(x, *args, **kwargs)) / dx
            return der

        return wrapper

    return der


@MAIN_DECORATOR(dx=0.1)
def f(x):
    return x * x * math.exp(x)


print(f(0))


@MAIN_DECORATOR()
def f(x):
    return x * x * math.exp(x)


print(f(0))


@MAIN_DECORATOR(dx=0.00000000000001)
def f(x):
    return x * x * math.exp(x)


print(f(0))

0.11051709180756479
0.00010001000050001668
1.00000000000001e-14


Ура! Все работает! Как видим, точность улучшается. Давайте на последок посмотрим, как бы мы это делали без `@`:

In [55]:
def MAIN_DECORATOR(dx=0.0001):
    def der(func):
        def wrapper(x, *args, **kwargs):
            der = (func(x + dx, *args, **kwargs) - func(x, *args, **kwargs)) / dx
            return der

        return wrapper

    return der


def f(x):
    return x * x * math.exp(x)


der_f = MAIN_DECORATOR(dx=0.00000001)(f)
print(der_f(0))

1.00000001e-08


## Проблема потери имени функции и ее документации

Рассмотрим простейший случай:

In [57]:
def decorator(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)

    return wrapper


def func():
    """Константа ноль"""
    return 0


print(func.__name__)
print(func.__doc__)

print("")


@decorator
def func():
    """Константа ноль"""
    return 0


print(func.__name__)
print(func.__doc__)

func
Константа ноль

wrapper
None


И замечаем здесь утерю возможно ценной информации. Хорошим тоном правильно было бы сделать так:

In [59]:
def decorator(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)

    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    return wrapper


def func():
    """Константа ноль"""
    return 0


print(func.__name__)
print(func.__doc__)

print("")


@decorator
def func():
    """Константа ноль"""
    return 0


print(func.__name__)
print(func.__doc__)

func
Константа ноль

func
Константа ноль


Это настолько стандартный подход, что он стандартизирован языком `Python`. Вместо этих двух строчек 
```python
wrapper.__name__ = func.__name__
wrapper.__doc__ = func.__doc__
```

Можем декорировать `wrapper` декоратором `wraps` из стандартной библиотеки:

In [60]:
from functools import wraps


def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)

    return wrapper


def func():
    """Константа ноль"""
    return 0


print(func.__name__)
print(func.__doc__)

print("")


@decorator
def func():
    """Константа ноль"""
    return 0


print(func.__name__)
print(func.__doc__)

func
Константа ноль

func
Константа ноль
